In [ ]:
import pandas as pd

stocks_df = pd.read_csv('PopularStockSymbols.csv')

In [ ]:
stocks_df

In [ ]:
symbols_df = stocks_df.set_index('Symbol')

In [ ]:
symbols_df

In [ ]:
import yfinance as yf
def YesterdayChangeP (v):
    YFSTK = yf.Ticker(v)
    stk_df = YFSTK.history(period="2d")
    return (stk_df.iloc[-1].Close-stk_df.iloc[-1].Open)/stk_df.iloc[-1].Open*100

In [ ]:
stocks_df['YesterdayChange'] = stocks_df.Symbol.apply(YesterdayChangeP)

In [ ]:
def LWeekChangeP (v):
    YFSTK = yf.Ticker(v)
    stk_df = YFSTK.history(period="8d")
    return (stk_df.iloc[-1].Close-stk_df.iloc[1].Open)/stk_df.iloc[1].Open*100

In [ ]:
stocks_df['LastWeekChange'] = stocks_df.Symbol.apply(LWeekChangeP)

In [ ]:
def LMonthChangeP (v):
    YFSTK = yf.Ticker(v)
    stk_df = YFSTK.history(period="31d")
    return (stk_df.iloc[-1].Close-stk_df.iloc[1].Open)/stk_df.iloc[1].Open*100

In [ ]:
stocks_df['LastMonthChange'] = stocks_df.Symbol.apply(LMonthChangeP)

In [ ]:
import time
import requests
Template = 'https://finnhub.io/api/v1/stock/recommendation?symbol={}&token=bsiqli7rh5rc8orbnkqg'

def Rating (v):
    Address = (Template.format(v))
    r = requests.get(Address)
    #time.sleep(1.01)
    #print(v)
    
    if not r.json():
        return 0
    else:
        ws = pd.Series(r.json()[0])
        Total = ws.buy + ws.hold + ws.sell + ws.strongBuy + ws.strongSell
        score = (ws.buy*2 + ws.hold - ws.sell*2 + ws.strongBuy*2.5 - ws.strongSell*2.5)/Total
        return score

In [ ]:
stocks_df['Rating'] = stocks_df.Symbol.apply(Rating)

In [ ]:
stocks_df.Rating = stocks_df.Rating.apply(lambda v: 0 if v<0 else v)

In [ ]:
stocks_df.set_index('Symbol', drop=True, inplace=True)
stocks_df.drop(columns=['Name'],inplace=True)

In [ ]:
stocks_df

In [ ]:
stocks_df.describe()

In [ ]:
columns = ['YesterdayChange','LastWeekChange','LastMonthChange','Rating']
import matplotlib.pyplot as plt

for col in columns:
    stocks_df[col].plot.hist()
    plt.show()
    stocks_df[col].plot.box(vert=False)
    plt.show()

In [ ]:
stocks_df_nor = (stocks_df - stocks_df.min())/(stocks_df.max() - stocks_df.min())*2-1

In [ ]:
stocks_df_nor.describe()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

SIL_results = pd.DataFrame(0.0, index =[4], 
                       columns= ['R{}'.format(i) for i in range(1,100)])

Clusterings =[]

for n_cluster in SIL_results.index:
    for col in SIL_results.columns:
        algort = KMeans(n_clusters=n_cluster).fit(stocks_df_nor)
        SIL_results.at[n_cluster,col] = silhouette_score(stocks_df_nor,algort.labels_)
        Clusterings.append(algort.labels_)
        # Inertia: Sum of distances of samples to their closest cluster center

SIL_results.transpose().plot.box()

SIL_results['Mean'] = SIL_results[['R{}'.format(i) for i in range(1,15)]].mean(axis=1)
SIL_results['Var'] = SIL_results[['R{}'.format(i) for i in range(1,15)]].var(axis=1)
SIL_results

In [ ]:
import numpy as np

memb = pd.Series(Clusterings[np.argmax(SIL_results)], index=stocks_df_nor.index)
memb

In [ ]:
stocks_df = stocks_df.join(pd.DataFrame(memb,columns=['Cluster']))
stocks_df

In [ ]:
Centroids_nor = pd.DataFrame(0.0, index = ['Cluster {}'.format(i) for i in range(4)],
                        columns = stocks_df_nor.columns)

for i in range(4):
    BM = memb==i
    Centroids_nor.iloc[i] = stocks_df_nor[BM].mean(axis=0)
    
Centroids_nor

In [ ]:
import seaborn as sns
for i in range(4):

    plt.figure(figsize=(10,4))
    Clust_name = 'Cluster ' + str(i)
    sns.heatmap(Centroids_nor.sort_values(Clust_name,axis=1).transpose(), linewidths=.5, annot=True, 
                cmap='coolwarm_r')
    plt.show()

In [ ]:
Cluster_names = ['Low Rate Low performance','High Rate Medium Perfomrance', 'High Rate High Performance','Mixed rate and performance']

In [ ]:
for clst in stocks_df.Cluster.unique():
    
    BM = stocks_df.Cluster == clst
    wdf = stocks_df[BM]
    
    print('Cluster Name: {}'.format(Cluster_names[clst]))
    print('Number of Members: {}'.format(len(wdf)))
    print(symbols_df.loc[wdf.index].Name)
    print('-----------------')
    
    